## Using other languages
Often, I hear that the biggest challenge of moving from another language to Julia is giving up all the codes you have written in other languages or your favorite packages from other languages. **This notebook is not about data science, but it's about your next data science project** (if you're working on a data science project in Julia and you want to use functionality from other langages). Here, we will specifically cover Python, R, and C.

### ⚫Python

In [1]:
using PyCall

You can import any python package...

In [2]:
math = pyimport("math")
math.sin(math.pi / 4) # returns ≈ 1/√2 = 0.70710678...

0.7071067811865476

Hint: you may need to do:
```
julia> using Conda
julia> Conda.add("networkx")
```
for the line below to work.

In [6]:
using Conda
Conda.add("networkx")

┌ Info: Running `conda install -y networkx` in root environment
└ @ Conda C:\Users\CaioSainVallio\.julia\packages\Conda\sNGum\src\Conda.jl:128


Solving environment: ...working... done

# All requested packages already installed.



In [15]:
python_networkx = pyimport("networkx")

LoadError: PyError (PyImport_ImportModule

The Python package networkx could not be imported by pyimport. Usually this means
that you did not install networkx in the Python version being used by PyCall.

PyCall is currently configured to use the Julia-specific Python distribution
installed by the Conda.jl package.  To install the networkx module, you can
use `pyimport_conda("networkx", PKG)`, where PKG is the Anaconda
package the contains the module networkx, or alternatively you can use the
Conda package directly (via `using Conda` followed by `Conda.add` etcetera).

Alternatively, if you want to use a different Python distribution on your
system, such as a system-wide Python (as opposed to the Julia-specific Python),
you can re-configure PyCall with that Python.   As explained in the PyCall
documentation, set ENV["PYTHON"] to the path/name of the python executable
you want to use, run Pkg.build("PyCall"), and re-launch Julia.

) <class 'ImportError'>
ImportError("cannot import name '

You can also write your own Python code as follows

In [16]:
py"""
import numpy
def find_best_fit_python(xvals,yvals):
    meanx = numpy.mean(xvals)
    meany = numpy.mean(yvals)
    stdx = numpy.std(xvals)
    stdy = numpy.std(yvals)
    r = numpy.corrcoef(xvals,yvals)[0][1]
    a = r*stdy/stdx
    b = meany - a*meanx
    return a,b
"""

In [17]:
xvals = repeat(1:0.5:10, inner=2)
yvals = 3 .+ xvals .+ 2 .* rand(length(xvals)) .-1
find_best_fit_python = py"find_best_fit_python"
a,b = find_best_fit_python(xvals,yvals)

(1.0473848715774257, 2.791312584721248)

If the above python code was in a file called `fit_linear.py`, you can call it as follows:
```
python_linear_fit = pyimport("fit_linear") 
python_linear_fit.find_best_fit_python(xvals,yvals)```

### ⚫R code

In [18]:
using RCall

`$` can switch to an `R` REPL from julia's REPL. We'll take a look...

In [19]:
# we can use the rcall function
r = rcall(:sum, Float64[1.0, 4.0, 6.0])

RObject{RealSxp}
[1] 11


In [20]:
typeof(r[1])

Float64

The `@rput` allows you to put julia variable in the `R` context.

In [21]:
z = 1
@rput z

1

In [22]:
r = R"z+z"

RObject{IntSxp}
[1] 2


In [23]:
r[1]

2

In [24]:
x = randn(10)

10-element Vector{Float64}:
 -0.573213859797309
 -0.26180497475656916
  2.4978431814760036
  1.3234550808507934
  0.8234130849177939
  0.26089034167394737
  0.085331862250728
 -1.5380694659555862
  0.1706925978564126
  0.35497187241811967

You can apply R functions on julia variables

In [25]:
@rimport base as rbase
rbase.sum([1, 2, 3])

RObject{IntSxp}
[1] 6


Hint: for the code below to work, you will need to type `$` in the REPL followed by:
```
install.packages("boot")
```
the `$` will enter you into the R REPL mode.

In [26]:
@rlibrary boot

In [27]:
R"t.test($x)"

RObject{VecSxp}

	One Sample t-test

data:  `#JL`$x
t = 0.9128, df = 9, p-value = 0.3851
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -0.4646934  1.0933954
sample estimates:
mean of x 
 0.314351 



The equivalent in Julia would be

In [28]:
using HypothesisTests
OneSampleTTest(x)

One sample t-test
-----------------
Population details:
    parameter of interest:   Mean
    value under h_0:         0
    point estimate:          0.314351
    95% confidence interval: (-0.4647, 1.0934)

Test summary:
    outcome with 95% confidence: fail to reject h_0
    two-sided p-value:           0.3851

Details:
    number of observations:   10
    t-statistic:              0.9127994285940914
    degrees of freedom:       9
    empirical standard error: 0.3443812104238518


### ⚫C code
Calling standard libraries is easy

In [29]:
t = ccall(:clock, Int32, ())

6049209

Can look at Python and C/C++ examples here: https://github.com/xorJane/Excelling-at-Julia-Basics-and-Beyond/blob/master/JuliaCon2019_Huda/Julia%20Wrappers.ipynb
```
ccall((:hello_world_repeated,"hello_world_lib.dylib"),
    Int64,
    (Int64,),
    10)
    ```

**Finally**, I would say that this is the only off-topic notebook in this course, and it's a topic that can be covered on its own in a standalone tutorial... Nevertheless, the goal of this notebook is to tell you that porting your code from Python, R, and C should be easy and straight forward in Julia. 

# 🥳 One cool finding

You can easily call Python, R, C, and Cpp code from Julia!